In [102]:
import RNA
import random

In [103]:
def penalize_barriers(i, j, k, l, d, arg_dict):
    penalty = 100 # WHAT SHOULD THIS BE???
    if d in [RNA.DECOMP_PAIR_IL, RNA.DECOMP_PAIR_HP, RNA.DECOMP_PAIR_ML]:
        ref = arg_dict['last']
        if i in ref.keys():
            if ref[i] == j:
                return 0
            else:
                return penalty
            
    return 0


def parse_dot_bracket(db):
    open_stack = []
    pairs = []
    for i, c in enumerate(db):
        if c == '(':
            open_stack.append(i)
        elif c == ')':
            pairs.append((open_stack.pop(), i))

    return(pairs)

def dict_dot_bracket(db):
    open_stack = []
    pairs = {}
    for i, c in enumerate(db):
        if c == '(':
            open_stack.append(i)
        elif c == ')':
            j = open_stack.pop()
            pairs[i+1] = j+1
            pairs[j+1] = i+1

    return(pairs)

In [104]:
def get_fc():
    #seq = ''.join(random.choices(['A', 'U', 'C', 'G'],[0.25, 0.25, 0.25, 0.25], k=20))
    seq = 'AGGACGCAACGCCGAACUAG'
    fc = RNA.fold_compound(seq)
    return fc

In [105]:
'''
AGGACGCAACGCCGAACUAG
(RNA.RNA.subopt_solution(structure=".((.((...)))).......", energy=-1.3),
 RNA.RNA.subopt_solution(structure="((..((......))..))..", energy=-0.5),
 RNA.RNA.subopt_solution(structure=".((........)).......", energy=-0.5))'''

'\nAGGACGCAACGCCGAACUAG\n(RNA.RNA.subopt_solution(structure=".((.((...)))).......", energy=-1.3),\n RNA.RNA.subopt_solution(structure="((..((......))..))..", energy=-0.5),\n RNA.RNA.subopt_solution(structure=".((........)).......", energy=-0.5))'

In [106]:
fc = get_fc()
print(fc.sequence)
fc.subopt(100)

AGGACGCAACGCCGAACUAG


(RNA.RNA.subopt_solution(structure=".((.((...)))).......", energy=-1.3),
 RNA.RNA.subopt_solution(structure="((..((......))..))..", energy=-0.5),
 RNA.RNA.subopt_solution(structure=".((........)).......", energy=-0.5))

In [107]:
fc = get_fc()
out = fc.mfe()
print(fc.sequence)
print(out[0], out[1])

AGGACGCAACGCCGAACUAG
.((.((...))))....... -1.2999999523162842


In [108]:
#           AAGCAUGCUACCGGAGGAGC
penalize = '((..((......))..))..'
parsed = dict_dot_bracket(penalize)
print(parsed)

fc2 = get_fc()
step_info = {
        'last' : dict_dot_bracket(penalize)
        }

fc2.sc_add_f(penalize_barriers)
fc2.sc_add_data(step_info)

out2 = fc2.mfe()
print(fc.sequence)
print(out[0], out[1])
print(out2[0], out2[1])

{13: 6, 6: 13, 14: 5, 5: 14, 17: 2, 2: 17, 18: 1, 1: 18}
AGGACGCAACGCCGAACUAG
.((.((...))))....... -1.2999999523162842
((..((......))..)).. -0.5


In [109]:
fc2.subopt(100)

(RNA.RNA.subopt_solution(structure="((..((......))..))..", energy=-0.5),
 RNA.RNA.subopt_solution(structure="....................", energy=0),
 RNA.RNA.subopt_solution(structure=".((........)).......", energy=0.5))

In [110]:
fc.eval_structure_verbose(out[0])

External loop                           :  -140
Interior loop (  2, 13) GC; (  3, 12) GC:  -330
Interior loop (  3, 12) GC; (  5, 11) CG:    40
Interior loop (  5, 11) CG; (  6, 10) GC:  -240
Hairpin  loop (  6, 10) GC              :   540


-1.2999999523162842

In [111]:
fc2.eval_structure_verbose(out[0])

External loop                           :  -140
Interior loop (  2, 13) GC; (  3, 12) GC:  -230
Interior loop (  3, 12) GC; (  5, 11) CG:    40
Interior loop (  5, 11) CG; (  6, 10) GC:  -140
Hairpin  loop (  6, 10) GC              :   640


1.7000000476837158

In [112]:
fc.eval_structure_verbose(out2[0])

External loop                           :   -20
Interior loop (  1, 18) AU; (  2, 17) GC:  -210
Interior loop (  2, 17) GC; (  5, 14) CG:   -50
Interior loop (  5, 14) CG; (  6, 13) GC:  -240
Hairpin  loop (  6, 13) GC              :   470


-0.5

In [113]:
fc2.eval_structure_verbose(out2[0])

External loop                           :   -20
Interior loop (  1, 18) AU; (  2, 17) GC:  -210
Interior loop (  2, 17) GC; (  5, 14) CG:   -50
Interior loop (  5, 14) CG; (  6, 13) GC:  -240
Hairpin  loop (  6, 13) GC              :   470


-0.5

In [114]:
fc.eval_structure_verbose('.((........)).......')

External loop                           :  -140
Interior loop (  2, 13) GC; (  3, 12) GC:  -330
Hairpin  loop (  3, 12) GC              :   420


-0.5

In [115]:
fc2.eval_structure_verbose('.((........)).......')

External loop                           :  -140
Interior loop (  2, 13) GC; (  3, 12) GC:  -230
Hairpin  loop (  3, 12) GC              :   420


0.5

In [116]:
import RNA
# Create a soft constraint function which returns a penalty when i is not paired to a specific j
def penalize_barriers(i, j, k, l, d, arg_dict):
    penalty = 100
    if d in [RNA.DECOMP_PAIR_IL, RNA.DECOMP_PAIR_HP, RNA.DECOMP_PAIR_ML]:
        ref = arg_dict['ref']
        if i in ref.keys():
            if ref[i] == j:
                return 0
            else:
                return penalty

            # There's no penalty for i being unpaired, but there should be
            
    return 0

# Parse a db string and return a dict of pairs
def dict_dot_bracket(db):
    open_stack = []
    pairs = {}
    for i, c in enumerate(db):
        if c == '(':
            open_stack.append(i)
        elif c == ')':
            j = open_stack.pop()
            pairs[i+1] = j+1
            pairs[j+1] = i+1

    return(pairs)
    
# According to this subopt, this sequence has the following structures:
# structure=".((.((...)))).......", energy=-1.3
# structure="((..((......))..))..", energy=-0.5
# structure=".((........)).......", energy=-0.5
seq = 'AGGACGCAACGCCGAACUAG'
fc = RNA.fold_compound(seq)

# Add a soft constraint penalizing structures different than a reference
penalize = '((..((......))..))..'
step_info = { 'ref' : dict_dot_bracket(penalize) }
fc.sc_add_f(penalize_barriers)
fc.sc_add_data(step_info)

out = fc.mfe()

print(fc.sequence)
print(out[0], out[1])

AGGACGCAACGCCGAACUAG
((..((......))..)).. -0.5


half-written response to Ronny which might not be needed
Thanks for the help, but I have one more question.  The example you gave there shows how to penalize certain base pairs, but is there a way to penalize a certain base being unpaired?  Here's a concrete example of what I'm trying to do:

```python
import RNA

# Create a soft constraint function which returns a penalty when i is not paired to a specific j
def penalize_barriers(i, j, k, l, d, arg_dict):
    penalty = 100
    if d in [RNA.DECOMP_PAIR_IL, RNA.DECOMP_PAIR_HP, RNA.DECOMP_PAIR_ML]:
        ref = arg_dict['ref']
        if i in ref.keys():
            if ref[i] == j:
                return 0
            else:
                return penalty

            # There's no penalty for i being unpaired, but there should be
            
    return 0

# Parse a db string and return a dict of pairs
def dict_dot_bracket(db):
    open_stack = []
    pairs = {}
    for i, c in enumerate(db):
        if c == '(':
            open_stack.append(i)
        elif c == ')':
            j = open_stack.pop()
            pairs[i] = j
            pairs[j] = i

    return(pairs)
    
# According to this subopt, this sequence has the following structures:
# structure=".((.((...)))).......", energy=-1.3
# structure="((..((......))..))..", energy=-0.5
# structure=".((........)).......", energy=-0.5
seq = 'AGGACGCAACGCCGAACUAG'
fc = RNA.fold_compound(seq)

# Add a soft constraint penalizing structures which break base pairs in a reference
reference = '((..((......))..))..'
step_info = { 'ref' : dict_dot_bracket(reference) }
fc.sc_add_f(penalize_barriers)
fc.sc_add_data(step_info)

out = fc.mfe()

print(fc.sequence)
print(out[0], out[1])
```

When I run this, I get 
```
AGGACGCAACGCCGAACUAG
.((........))....... -0.5
```
Which is missing 3 pairs which exist in the reference (the bases which are formed are good).  Additionally, when I run `fc.eval_structure_verbose(reference)`, I would expect it to return the same value as it did without any constraints (-0.5), 